In [1]:
'''각종 라이브러리 import'''
# 인공지능 라이브러리
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import numpy as np
# 파이썬 라이브러리
import os
import time
# 이미지 관련 라이브러리
import glob
from PIL import Image

In [2]:
# 악성코드 데이터 불러오기
'''
normal_image = glob.glob(r"D:\OneDrive - 한림대학교\바탕 화면\Working Space\Capstone\dataset\image\normal\*thumb.png ")

# 일반 데이터를 저장할 배열 생성
nor_len = 800 #len(os.listdir(r"D:\OneDrive - 한림대학교\바탕 화면\Working Space\Capstone\dataset\image\normal\ "))
normal_input = np.zeros((nor_len,28,28,1), dtype="float32")
normal_target = np.zeros((nor_len), dtype="uint8")

# 이미지를 불러온 후, 28*28로 조정
cnt = 0
print("일반 파일 처리중")
for i in normal_image:
    #print(cnt,"번째 처리중: ",i)
    im = Image.open(i).convert("L")
    out = im.resize((28,28))
    normal_input[cnt,:,:,0] = out
    normal_target[cnt] = 0 # normal = 0
    cnt = cnt + 1
'''
# 악성코드 데이터 불러오기
malware_image = glob.glob(r"D:\OneDrive - 한림대학교\바탕 화면\Working Space\Capstone\dataset\image\malware\*thumb.png")

# 악성코드 데이터를 저장할 배열 생성
mal_len = 39284 #len(os.listdir(r'D:\OneDrive - 한림대학교\바탕 화면\Working Space\Capstone\dataset\image\malware\ ')) # 샘플의 갯수
malware_input = np.zeros((mal_len,28,28,1), dtype="float32")
malware_target = np.zeros(mal_len, dtype="uint8")

# 이미지를 불러온 후, 28*28로 조정
cnt = 0
print("악성 파일 처리중")
for i in malware_image:
    #print(cnt,"번째 처리중: ",i)
    im = Image.open(i).convert("L")
    out = im.resize((28,28))
    malware_input[cnt,:,:,0] = out
    malware_target[cnt] = 1 # malware = 1
    cnt = cnt + 1
    
# 데이터 합치기

input_data = malware_input
target_data = malware_target

#input_data = np.concatenate((normal_input, malware_input), axis=0)
#target_data = np.concatenate((normal_target, malware_target),axis=0)

악성 파일 처리중


In [3]:
input_data.shape

(39284, 28, 28, 1)

In [4]:
# 테스트셋 분리하기
train_input, test_input, train_target, test_target = train_test_split(input_data, target_data, test_size=0.3)

train_scaled = train_input.reshape(-1,28,28,1) / 255.0
test_scaled = test_input.reshape(-1,28,28,1) / 255.0

In [5]:
'''합성곱 신경망 만들기'''
with tf.device('/CPU:0'):
    # 합성곱 신경망
    model = keras.Sequential()

    # 1st Layer
    model.add(keras.layers.Conv2D(64, kernel_size = (3,3), activation = 'relu', padding = 'same', input_shape = (28,28,1), strides= (1,1)))
    model.add(keras.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = 'same'))
    model.add(keras.layers.Dropout(0.8))

    # 2nd Layer
    model.add(keras.layers.Conv2D(128, kernel_size = (3,3), activation = 'relu', padding = 'same', strides= (1,1)))
    model.add(keras.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = 'same'))
    model.add(keras.layers.Dropout(0.2))

    # 3rd Layer
    model.add(keras.layers.Conv2D(256, kernel_size = (3,3), activation = 'relu', padding = 'same', strides= (1,1)))
    model.add(keras.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = 'same'))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Flatten())

    # 4th Layer
    model.add(keras.layers.Dense(4096, activation = 'relu'))
    model.add(keras.layers.Dropout(0.2))

    # 5th Layer
    model.add(keras.layers.Dense(1, activation = 'softmax'))

    # 모델에 대한 시각화
    model.summary() # 모델의 구조 보기
    keras.utils.plot_model(model) # 모델의 구조를 그림으로 표현
    keras.utils.plot_model(model, show_shapes=True) # 모델의 구조를 그림으로 표현하며, 입력값의 shape도 출력

    # Model Compile
    model.compile(optimizer = 'adam', loss = keras.losses.BinaryCrossentropy(), metrics='accuracy')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 128)        0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 7, 7, 128)         0

In [8]:
'''모델 컴파일과 훈련'''
with tf.device('/CPU:0'):
    k = 5
    num_epochs = 100
    batch = 10

    num_val_samples = len(train_input) // k

    start_time = time.time()
    for i in range(k):
        print(i,'번째 Fold 처리중')

        partial_val_input = train_scaled[i * num_val_samples : (i+1) * num_val_samples]
        partial_val_target = train_target[i * num_val_samples : (i+1) * num_val_samples]

        partial_train_input = np.concatenate(
            [train_scaled[:i * num_val_samples],
            train_scaled[(i+1) * num_val_samples:]],
            axis=0
        )
        partial_train_target = np.concatenate(
            [train_target[:i * num_val_samples],
            train_target[(i+1) * num_val_samples:]],
            axis=0
        )

        checkpoint_cb = keras.callbacks.ModelCheckpoint('best-cnn-model.h5', save_best_only=True) # Declare ModelCheckPoint Callback Object
        early_stopping_cb = keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True) # Declare EarlyStopping Callback Object
        history = model.fit(
            partial_train_input, partial_train_target, epochs=num_epochs, batch_size=batch,
            validation_data = (partial_val_input, partial_val_target),
            callbacks=[checkpoint_cb, early_stopping_cb], verbose = 1) # Train Model

    model.evaluate(test_scaled, test_target) # Model Evaluation
    end_time = time.time()
    print("실행시간: ", end_time-start_time)

0 번째 Fold 처리중
Epoch 1/100
2200/2200 [==============================] - 121s 55ms/step - loss: 4.3927e-18 - accuracy: 1.0000 - val_loss: 5.9084e-15 - val_accuracy: 1.0000
Epoch 2/100
2200/2200 [==============================] - 119s 54ms/step - loss: 2.1736e-18 - accuracy: 1.0000 - val_loss: 5.9084e-15 - val_accuracy: 1.0000
Epoch 3/100
2200/2200 [==============================] - 119s 54ms/step - loss: 4.4514e-19 - accuracy: 1.0000 - val_loss: 5.9084e-15 - val_accuracy: 1.0000
1 번째 Fold 처리중
Epoch 1/100
2200/2200 [==============================] - 121s 55ms/step - loss: 2.5651e-17 - accuracy: 1.0000 - val_loss: 5.7601e-12 - val_accuracy: 1.0000
Epoch 2/100
2200/2200 [==============================] - 123s 56ms/step - loss: 4.0911e-18 - accuracy: 1.0000 - val_loss: 5.7601e-12 - val_accuracy: 1.0000
Epoch 3/100
2200/2200 [==============================] - 123s 56ms/step - loss: 2.1931e-18 - accuracy: 1.0000 - val_loss: 5.7601e-12 - val_accuracy: 1.0000
2 번째 Fold 처리중
Epoch 1/100
2200/2200 

In [ ]:
# 학습셋과 검증셋의 손실값 시각화 비교
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label = 'Train')
plt.plot(history.history['val_loss'], label = 'Val')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
# 학습셋과 검증셋의 metrics 시각화 비교
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'], label = 'Train')
plt.plot(history.history['val_accuracy'], label = 'Val')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
# 이미지로 출력해보기
plt.imshow(test_scaled[0].reshape(28,28), cmap='gray_r') # 기존 데이터는 합성곱 연산을 위해 임의적으로 3차원을 만들었으나, 원래 데이터는 흑백이므로 색상데이터가 필요없어 2차원 데이터로 변환
plt.show()
print(test_scaled.shape)

# 이미지에 대한 예측 확인하기
preds = model.predict(test_scaled) # Keras의 fit()/predict()/evaluate()는 모두 입력의 첫번째 차원이 배치 차원일 것으로 기대함. 따라서 결과 값은 (1, 28, 28, 1)과 같이 더 추가됨. 따라서 슬라이싱을 통해 원소 접근해야 함.
print(preds) # 모델의 출력층의 뉴런 갯수에 따라 10개의 클래스에 대한 예측 확률을 출력함 # 여기서는 9번째 값이 1에 가까우므로, 해당 이미지는 9번째 클래스라고 추측가능.

# Bar로 클래스 구분짓기
plt.bar(0,1, preds)
plt.xlabel('class')
plt.ylabel('prob.')
plt.show()